In [1]:
import codecs
from collections import Counter
import string
from Crypto.Cipher import AES
from Crypto import Random
import random as rnd
import time
import hashlib
import requests

# those variables are used later in several problems so I just initialize them here.
secret_key = Random.get_random_bytes(16)
iv_secret = Random.get_random_bytes(16)
secret_key_16_bits = Random.get_random_bytes(2)

Copied from cryptopals.com
## Crypto Challenge Set 1

This is the qualifying set. We picked the exercises in it to ramp developers up gradually into coding cryptography, but also to verify that we were working with people who were ready to write code.

This set is relatively easy. With one exception, most of these exercises should take only a couple minutes. But don't beat yourself up if it takes longer than that. It took Alex two weeks to get through the set!

If you've written any crypto code in the past, you're going to feel like skipping a lot of this. Don't skip them. At least two of them (we won't say which) are important stepping stones to later attacks. 


## Convert hex to base64

The string:
```
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
```
Should produce:
```
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
```
So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

Cryptopals Rule
```
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.
```

In [2]:
## Challenge 1 convert from hex to base64
## This one is pretty straightforward
input = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
raw = codecs.decode(input, "hex")
result = codecs.encode(raw,"base64")
expected_result = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
result.strip().decode('ascii') == expected_result

True


## Fixed XOR

Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965

... should produce:

746865206b696420646f6e277420706c6179



In [3]:
## Challenge 2 FIxed XOR
input1 = codecs.decode("1c0111001f010100061a024b53535009181c","hex")
input2 = codecs.decode("686974207468652062756c6c277320657965","hex")

def xor_string(string1,string2):
    """ Receive two bytes strings and return the xor in them
        The length of the output is the same as the shortest string
    """
    return bytes(a ^ b for a, b in zip(string1, string2))

expected_result = "746865206b696420646f6e277420706c6179"

codecs.encode(xor_string(input1,input2),"hex").strip().decode('ascii') == expected_result

True

## Single-byte XOR cipher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

Achievement Unlocked

You now have our permission to make "ETAOIN SHRDLU" jokes on Twitter.


In [4]:
## Challenge 3 xor single byte
input1 = codecs.decode("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736","hex")

#Here I'm using as an score of a piece of English plaintext not the character frequency but rather the fact
#that the correct english phrase is more likely to have the most amount of words, this isn't valid in all cases,but worked in this one
biggest = 0;
biggest_character = '';
for character in range(0,128):
    output = xor_string(input1,bytes(chr(character)*len(input1),'ascii'))
    if len(output.decode('ascii').split(' '))>biggest:
        biggest = len(output.decode('ascii').split(' '))
        biggest_character = character
        
output = xor_string(input1,bytes(chr(biggest_character)*len(input1),'ascii'))
output

b"Cooking MC's like a pound of bacon"

## Detect single-character XOR

One of the 60-character strings in this file has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [5]:
## Challenge 4 find encoded string in file

lines = open('4.txt', 'r').readlines()

## I'm gonna use the same logic from the 3rd problem and see the line with the most amont of words
longest_line =""
global_longest = 0;

for line in lines:
    line_raw = codecs.decode(line.strip(),'hex')
    biggest = 0;
    biggest_character = 0;
    for character in range(0,128):
        output = xor_string(line_raw,bytes(len(line_raw)*chr(character),'ascii'))
        flag = False;
        for i in output:
            if i>=128: # if output is not in ascii then it's not an english sentence
                flag = True;
        
        if flag:
            continue
        
        if len(output.decode('ascii').split(' '))>biggest: ## compare between characters in the line
            biggest = len(output.decode('ascii').split(' '))
            biggest_character = character;
            
        if biggest>global_longest: ## compare between all lines
            global_longest=biggest;
            longest_line = output;

print(longest_line)

b'Now that the party is jumping\n'


## Implement repeating-key XOR

Here is the opening stanza of an important work of the English language:
```
Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
```
Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272
a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f

Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.


In [6]:
## Challenge 5 repeating key xor

#def repeating_xor(input1,key):
#    output = b''
#    while len(input1)>0:
#        output += xor_string(input1,key) 
#        input1 = input1[len(key):]
#    return output

# this one is a little bit faster, although the other function is easier to read and uses previous functions
def repeating_xor(input1,key):
    output=[]
    for s in range(0,len(input1)):
        output.append(input1[s] ^ key[s % len(key)])
    return bytes(output)


input1 = "Burning 'em, if you ain't quick and nimble"
key = "ICE"
input2 = "I go crazy when I hear a cymbal"

input3 = input1+ '\n' +input2;
result1 = repeating_xor(bytes(input1,'ascii'),bytes(key,'ascii'))
result2 = repeating_xor(bytes(input2,'ascii'),bytes(key,'ascii'))
result3 = repeating_xor(bytes(input3,'ascii'),bytes(key,'ascii'))

expected1 = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272"
expected2 = "a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

codecs.encode(result3,'hex').decode('ascii') == expected1 + expected2

True